# Atividade • Data Science
<hr>

## Data Discovery

### Sumário

<p> Número de clientes: 105,508

Número de produtos: 5,850

Número de transações: 192,544 </p>


### Dicionário

- **item_id:** ID exclusivo do produto

- **weight:** medição de peso do cliente

- **rented for:** roupas para esse fim que foram alugadas para

- **body type:** tipo de corpo do cliente

- **review_text:** review given by the customer

- **review_summary:** avaliação dada pelo cliente

- **size:** o tamanho padronizado do produto

- **rating:** classificação do produto

- **age:** idade do cliente

- **category:** a categoria do produto

- **bust size:** medição do busto do cliente

- **height:** altura do cliente

- **fit:** ajuste de feedback

- **user_id:** um ID exclusivo para o cliente

- **review_date**: data em que a resenha foi escrita

## Objetivos
<p> 1. Produza um gráfico da distribuição de peso (histograma) dos clientes (<strong>em Kg</strong>)</p>
<p> 2. Qual motivo de aluguel (“rented for”) tem o maior número absoluto de “fits”?  Quantos “fits” há para este motivo de aluguel?</p>
<p>3. Qual motivo de aluguel (“rented for”) tem o maior número relativo de “fits”? Qual é o percentual de “fits” para este motivo de aluguel?</p>
<p>4. Produza uma estatística de palavras mais usadas nas avaliações (“review_text”);</p>
<p>5. Que outra estatística ou insight você poderia extrair destes dados?</p>

## Preparando o Ambiente

### Distribuição Linux

In [5]:
! lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


### Versão do Python

In [3]:
! python -V

Python 3.10.12


### Instalando as libs

In [2]:
! pip install -r "../requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 823.2 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 6.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e5faf3fa0028d7b41e987bdd67adf78ff90cf7bf840219650506b7ef089afd01
  Stored in directory: /home/dgamorim/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


### Instalando o JDK

- Execute no terminal o seguinte comando:
  - ```sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null```

In [8]:
! java -version

openjdk version "1.8.0_382"
OpenJDK Runtime Environment (build 1.8.0_382-8u382-ga-1~22.04.1-b05)
OpenJDK 64-Bit Server VM (build 25.382-b05, mixed mode)


### Download do Spark

In [10]:
! wget -q -P "../spark" "https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz"

^C


In [14]:
! tar xf "../spark/spark-3.5.0-bin-hadoop3.tgz" -C "../spark"

In [15]:
! rm -rf "../spark/spark-3.5.0-bin-hadoop3.tgz"

### Importando as bibliotecas

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

### Instanciando o Spark

In [16]:
! export JAVA_HOME=$(readlink -f $(which java))
! export SPARK_HOME="$(readlink -f ../)/spark/spark-3.5.0-bin-hadoop3"

In [17]:
import findspark
findspark.init()

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Atividade Data Science - Multiplan") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/10/24 08:56:02 WARN Utils: Your hostname, DESKTOP-FLROA7K resolves to a loopback address: 127.0.1.1; using 172.23.17.233 instead (on interface eth0)
23/10/24 08:56:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/24 08:56:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
spark